In [50]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import re
import os

In [51]:
from sklearn import metrics
import sklearn
from sklearn.model_selection  import train_test_split
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.neural_network import MLPClassifier

In [52]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow import keras
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import Callback

In [100]:
data = pd.read_csv('D:/Dcode/Data/football/train.csv')
datat = pd.read_csv('D:/Dcode/Data/football/test.csv')
datatt = pd.read_csv('D:/Dcode/Data/football/test.csv')
data.head()

,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,is_goal,...,lat/lng,type_of_shot,type_of_combined_shot,match_id,team_id,remaining_min.1,power_of_shot.1,knockout_match.1,remaining_sec.1,distance_of_shot.1
0,12.0,-157.0,0.0,10.0,1.0,0.0,2000-01,22.0,35.0,0.0,...,"45.539131, -122.651648",shot - 45,NaN,20000012,1610612747,10.00,1.0,28.800,22.0000,35.0
1,35.0,-101.0,135.0,7.0,1.0,0.0,2000-01,45.0,36.0,1.0,...,"45.539131, -122.651648",shot - 25,NaN,20000012,1610612747,92.64,1.0,0.000,63.7216,54.4
2,43.0,138.0,175.0,6.0,1.0,0.0,2000-01,52.0,42.0,0.0,...,"45.539131, -122.651648",NaN,shot - 3,20000012,1610612747,NaN,1.0,122.608,52.0000,42.0
3,155.0,0.0,0.0,NaN,2.0,0.0,2000-01,19.0,20.0,1.0,...,"45.539131, -122.651648",NaN,shot - 1,20000012,1610612747,42.64,2.0,0.000,19.0000,20.0
4,244.0,-145.0,-11.0,9.0,3.0,0.0,NaN,32.0,34.0,0.0,...,"45.539131, -122.651648",shot - 17,NaN,20000012,1610612747,9.00,3.0,0.000,NaN,34.0


In [54]:
data = data.loc[:,~data.columns.duplicated()]
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
datat = datat.loc[:,~datat.columns.duplicated()]
datat = datat.loc[:, ~datat.columns.str.contains('^Unnamed')]

In [55]:
toDrop = [
    'power_of_shot.1',
    'remaining_min.1',
    'remaining_sec.1',
    'distance_of_shot.1',
    'knockout_match.1',
    'date_of_game',
    'match_id',
    'team_id',
    'shot_basics',
    'range_of_shot',
    'match_event_id'
]
data.drop(toDrop, axis=1, inplace=True)
datat.drop(toDrop, axis=1, inplace=True)
data.shape

(19604, 16)

In [56]:
data = data[np.isfinite(data['is_goal'])]
data.head()

,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,is_goal,area_of_shot,team_name,home/away,shot_id_number,lat/lng,type_of_shot,type_of_combined_shot
0,-157.0,0.0,10.0,1.0,0.0,2000-01,22.0,35.0,0.0,Left Side(L),Manchester United,MANU @ POR,2.0,"45.539131, -122.651648",shot - 45,NaN
1,-101.0,135.0,7.0,1.0,0.0,2000-01,45.0,36.0,1.0,Left Side Center(LC),Manchester United,NaN,3.0,"45.539131, -122.651648",shot - 25,NaN
2,138.0,175.0,6.0,1.0,0.0,2000-01,52.0,42.0,0.0,Right Side Center(RC),Manchester United,MANU @ POR,4.0,"45.539131, -122.651648",NaN,shot - 3
3,0.0,0.0,NaN,2.0,0.0,2000-01,19.0,20.0,1.0,Center(C),NaN,MANU @ POR,5.0,"45.539131, -122.651648",NaN,shot - 1
4,-145.0,-11.0,9.0,3.0,0.0,NaN,32.0,34.0,0.0,Left Side(L),Manchester United,MANU @ POR,6.0,"45.539131, -122.651648",shot - 17,NaN


In [57]:
mdata=data
toDrop = [
    'team_name',
    'home/away',
    'shot_id_number',
    #'lat/lng',
   # 'location_x',
  #  'location_y',
    'game_season'
]
data.drop(toDrop, axis=1, inplace=True)
datat.drop(toDrop, axis=1, inplace=True)

In [58]:
data['Lat'], data['Lon'] = data['lat/lng'].str.split(', ').str
data.Lat=data.Lat.astype(float)
data.Lon=data.Lon.astype(float)
data.drop('lat/lng', axis=1, inplace=True)
datat['Lat'], datat['Lon'] = datat['lat/lng'].str.split(', ').str
datat.Lat=datat.Lat.astype(float)
datat.Lon=datat.Lon.astype(float)
datat.drop('lat/lng', axis=1, inplace=True)

In [59]:
data.shape

(19604, 13)

In [60]:
datat.shape

(3575, 12)

In [61]:
data=pd.get_dummies(data, columns=["area_of_shot"], prefix=["area"])
datat=pd.get_dummies(datat, columns=["area_of_shot"], prefix=["area"])

In [62]:
data.shape

(19604, 18)

In [63]:
datat.shape

(3575, 17)

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19604 entries, 0 to 19603
Data columns (total 18 columns):
location_x                    18701 non-null float64
location_y                    18613 non-null float64
remaining_min                 18604 non-null float64
power_of_shot                 18673 non-null float64
knockout_match                18670 non-null float64
remaining_sec                 18592 non-null float64
distance_of_shot              18585 non-null float64
is_goal                       19604 non-null float64
type_of_shot                  9846 non-null object
type_of_combined_shot         9758 non-null object
Lat                           18605 non-null float64
Lon                           18605 non-null float64
area_Center(C)                19604 non-null uint8
area_Left Side Center(LC)     19604 non-null uint8
area_Left Side(L)             19604 non-null uint8
area_Mid Ground(MG)           19604 non-null uint8
area_Right Side Center(RC)    19604 non-null uint8
area

In [65]:
data.remaining_min.isnull().sum()
data['remaining_min']=data['remaining_min'].replace(np.NaN, int(data['remaining_min'].mean()))
data['power_of_shot']=data['power_of_shot'].replace(np.NaN, int(data['power_of_shot'].mean()))
data['knockout_match']=data['knockout_match'].replace(np.NaN, int(data['knockout_match'].mean()))
data['remaining_sec']=data['remaining_sec'].replace(np.NaN, int(data['remaining_sec'].mean()))
data['distance_of_shot']=data['distance_of_shot'].replace(np.NaN, int(data['distance_of_shot'].mean()))

data['location_x']=data['location_x'].replace(np.NaN, int(data['location_x'].mean()))
data['location_y']=data['location_y'].replace(np.NaN, int(data['location_y'].mean()))
data['Lat']=data['Lat'].replace(np.NaN, int(data['Lat'].mean()))
data['Lon']=data['Lon'].replace(np.NaN, int(data['Lon'].mean()))

data['is_goal']=data['is_goal'].replace(np.NaN, int(data['is_goal'].mean()))
data['type_of_shot']=data['type_of_shot'].replace(np.NaN, "shot - NaN")
data['type_of_combined_shot']=data['type_of_combined_shot'].replace(np.NaN, "shot - NaN")

In [66]:
datat.remaining_min.isnull().sum()
datat['remaining_min']=datat['remaining_min'].replace(np.NaN, int(datat['remaining_min'].mean()))
datat['power_of_shot']=datat['power_of_shot'].replace(np.NaN, int(datat['power_of_shot'].mean()))
datat['knockout_match']=datat['knockout_match'].replace(np.NaN, int(datat['knockout_match'].mean()))
datat['remaining_sec']=datat['remaining_sec'].replace(np.NaN, int(datat['remaining_sec'].mean()))
datat['distance_of_shot']=datat['distance_of_shot'].replace(np.NaN, int(datat['distance_of_shot'].mean()))

datat['location_x']=datat['location_x'].replace(np.NaN, int(datat['location_x'].mean()))
datat['location_y']=datat['location_y'].replace(np.NaN, int(datat['location_y'].mean()))
datat['Lat']=datat['Lat'].replace(np.NaN, int(datat['Lat'].mean()))
datat['Lon']=datat['Lon'].replace(np.NaN, int(datat['Lon'].mean()))

#datat['is_goal']=datat['is_goal'].replace(np.NaN, int(datat['is_goal'].mean()))
datat['type_of_shot']=datat['type_of_shot'].replace(np.NaN, "shot - NaN")
datat['type_of_combined_shot']=datat['type_of_combined_shot'].replace(np.NaN, "shot - NaN")

In [67]:
data['remaining_min'] = data['remaining_min'].astype(int)
data['power_of_shot'] = data['power_of_shot'].astype(int)
data['knockout_match'] = data['knockout_match'].astype(int)
data['remaining_sec'] = data['remaining_sec'].astype(int)
data['is_goal'] = data['is_goal'].astype(int)

data['location_x'] = data['location_x'].astype(int)
data['location_y'] = data['location_y'].astype(int)

data['distance_of_shot'] = data['distance_of_shot'].astype(int)

In [69]:
datat['remaining_min'] = datat['remaining_min'].astype(int)
datat['power_of_shot'] = datat['power_of_shot'].astype(int)
datat['knockout_match'] = datat['knockout_match'].astype(int)
datat['remaining_sec'] = datat['remaining_sec'].astype(int)
#datat['is_goal'] = datat['is_goal'].astype(int)

datat['location_x'] = datat['location_x'].astype(int)
datat['location_y'] = datat['location_y'].astype(int)

datat['distance_of_shot'] = datat['distance_of_shot'].astype(int)

In [70]:
data.tail()

,location_x,location_y,remaining_min,power_of_shot,knockout_match,remaining_sec,distance_of_shot,is_goal,type_of_shot,type_of_combined_shot,Lat,Lon,area_Center(C),area_Left Side Center(LC),area_Left Side(L),area_Mid Ground(MG),area_Right Side Center(RC),area_Right Side(R)
19599,-169,121,10,2,0,41,40,0,shot - NaN,shot - 3,38.567296,-121.456638,0,0,1,0,0,0
19600,0,0,5,2,0,23,20,1,shot - NaN,shot - 5,38.567296,-121.456638,1,0,0,0,0,0
19601,-162,3,4,2,0,34,36,1,shot - 56,shot - NaN,38.567296,-121.456638,0,0,1,0,0,0
19602,7,0,2,2,0,26,20,1,shot - NaN,shot - 4,38.567296,-121.456638,1,0,0,0,0,0
19603,0,0,0,3,0,42,20,0,shot - NaN,shot - 4,40.000000,-85.000000,1,0,0,0,0,0


In [71]:
data=pd.get_dummies(data, columns=["type_of_shot"], prefix=["type_of_shot"])
data=pd.get_dummies(data, columns=["type_of_combined_shot"], prefix=["type_of_combined_shot"])
datat=pd.get_dummies(datat, columns=["type_of_shot"], prefix=["type_of_shot"])
datat=pd.get_dummies(datat, columns=["type_of_combined_shot"], prefix=["type_of_combined_shot"])
data

,location_x,location_y,remaining_min,power_of_shot,knockout_match,remaining_sec,distance_of_shot,is_goal,Lat,Lon,...,type_of_shot_shot - 8,type_of_shot_shot - 9,type_of_shot_shot - NaN,type_of_combined_shot_shot - 0,type_of_combined_shot_shot - 1,type_of_combined_shot_shot - 2,type_of_combined_shot_shot - 3,type_of_combined_shot_shot - 4,type_of_combined_shot_shot - 5,type_of_combined_shot_shot - NaN
0,-157,0,10,1,0,22,35,0,45.539131,-122.651648,...,0,0,0,0,0,0,0,0,0,1
1,-101,135,7,1,0,45,36,1,45.539131,-122.651648,...,0,0,0,0,0,0,0,0,0,1
2,138,175,6,1,0,52,42,0,45.539131,-122.651648,...,0,0,1,0,0,0,1,0,0,0
3,0,0,4,2,0,19,20,1,45.539131,-122.651648,...,0,0,1,0,1,0,0,0,0,0
4,-145,-11,9,3,0,32,34,0,45.539131,-122.651648,...,0,0,0,0,0,0,0,0,0,1
5,0,0,8,2,0,52,20,1,45.539131,-122.651648,...,0,0,1,0,0,0,0,1,0,0
6,-65,91,6,3,0,12,32,1,45.539131,-122.651648,...,0,0,0,0,0,0,0,0,0,1
7,-33,91,3,3,0,36,32,0,45.539131,-122.651648,...,0,0,0,0,0,0,0,0,0,1
8,121,127,11,1,0,0,37,1,42.982923,-71.446094,...,0,0,1,0,0,0,1,0,0,0
9,-67,91,7,1,0,28,33,1,42.982923,-71.446094,...,0,0,0,0,0,0,0,0,0,1


In [72]:
data.shape

(19604, 81)

In [73]:
datat.shape

(3575, 80)

In [74]:
y=data['is_goal']
x=data
x.drop('is_goal', axis=1, inplace=True)
print(y)

0        0
1        1
2        0
3        1
4        0
5        1
6        1
7        0
8        1
9        1
10       0
11       0
12       0
13       1
14       0
15       0
16       1
17       1
18       1
19       0
20       0
21       0
22       0
23       0
24       1
25       0
26       1
27       1
28       0
29       1
        ..
19574    1
19575    1
19576    0
19577    0
19578    0
19579    0
19580    0
19581    1
19582    1
19583    0
19584    1
19585    1
19586    1
19587    0
19588    0
19589    1
19590    1
19591    0
19592    1
19593    0
19594    1
19595    0
19596    1
19597    0
19598    0
19599    0
19600    1
19601    1
19602    1
19603    0
Name: is_goal, Length: 19604, dtype: int32


In [75]:
data['location_x']=data['location_x']/250
data['location_y']=data['location_y']/350
datat['location_x']=datat['location_x']/250
datat['location_y']=datat['location_y']/350
data

,location_x,location_y,remaining_min,power_of_shot,knockout_match,remaining_sec,distance_of_shot,Lat,Lon,area_Center(C),...,type_of_shot_shot - 8,type_of_shot_shot - 9,type_of_shot_shot - NaN,type_of_combined_shot_shot - 0,type_of_combined_shot_shot - 1,type_of_combined_shot_shot - 2,type_of_combined_shot_shot - 3,type_of_combined_shot_shot - 4,type_of_combined_shot_shot - 5,type_of_combined_shot_shot - NaN
0,-0.628,0.000000,10,1,0,22,35,45.539131,-122.651648,0,...,0,0,0,0,0,0,0,0,0,1
1,-0.404,0.385714,7,1,0,45,36,45.539131,-122.651648,0,...,0,0,0,0,0,0,0,0,0,1
2,0.552,0.500000,6,1,0,52,42,45.539131,-122.651648,0,...,0,0,1,0,0,0,1,0,0,0
3,0.000,0.000000,4,2,0,19,20,45.539131,-122.651648,1,...,0,0,1,0,1,0,0,0,0,0
4,-0.580,-0.031429,9,3,0,32,34,45.539131,-122.651648,0,...,0,0,0,0,0,0,0,0,0,1
5,0.000,0.000000,8,2,0,52,20,45.539131,-122.651648,1,...,0,0,1,0,0,0,0,1,0,0
6,-0.260,0.260000,6,3,0,12,32,45.539131,-122.651648,0,...,0,0,0,0,0,0,0,0,0,1
7,-0.132,0.260000,3,3,0,36,32,45.539131,-122.651648,1,...,0,0,0,0,0,0,0,0,0,1
8,0.484,0.362857,11,1,0,0,37,42.982923,-71.446094,0,...,0,0,1,0,0,0,1,0,0,0
9,-0.268,0.260000,7,1,0,28,33,42.982923,-71.446094,0,...,0,0,0,0,0,0,0,0,0,1


In [76]:
print(x.shape)
y.shape

(19604, 80)


(19604,)

In [77]:
y_binary = to_categorical(y)
y_binary.dtype

dtype('float32')

In [78]:
print(x.shape)
y.shape

(19604, 80)


(19604,)

In [89]:
model = tf.keras.Sequential([
    layers.Dense(10, activation='relu', input_shape=(80,)),
    layers.Dense(20, activation='relu'),
    layers.Dense(2, activation='softmax')])

In [90]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [91]:
reducer = keras.callbacks.ReduceLROnPlateau(monitor="val_loss")
earlystopper = keras.callbacks.EarlyStopping(monitor='val_loss')

In [92]:
with tf.device('/device:GPU:0'):
    model.fit(x,y, epochs=100, batch_size=30, validation_split=0.3, callbacks=[reducer])

Train on 13722 samples, validate on 5882 samples
Epoch 1/100
13722/13722 [==============================] - 2s 129us/sample - loss: 0.8127 - acc: 0.5590 - val_loss: 0.6765 - val_acc: 0.5826
Epoch 2/100
13722/13722 [==============================] - 1s 106us/sample - loss: 0.6748 - acc: 0.5833 - val_loss: 0.7146 - val_acc: 0.5801
Epoch 3/100
13722/13722 [==============================] - 1s 108us/sample - loss: 0.6736 - acc: 0.5905 - val_loss: 0.6549 - val_acc: 0.6190
Epoch 4/100
13722/13722 [==============================] - 1s 109us/sample - loss: 0.6765 - acc: 0.5830 - val_loss: 0.6568 - val_acc: 0.6251
Epoch 5/100
13722/13722 [==============================] - 1s 105us/sample - loss: 0.6646 - acc: 0.6021 - val_loss: 0.6622 - val_acc: 0.6233
Epoch 6/100
13722/13722 [==============================] - 2s 111us/sample - loss: 0.6670 - acc: 0.5945 - val_loss: 0.6672 - val_acc: 0.5843
Epoch 7/100
13722/13722 [==============================] - 2s 112us/sample - loss: 0.6637 - acc: 0.6022 -

In [93]:
result = model.predict(datat, batch_size=32)
result

array([[0.45764974, 0.5423503 ],
       [0.55135643, 0.44864354],
       [0.03857527, 0.96142477],
       ...,
       [0.62677145, 0.37322855],
       [0.6745697 , 0.32543024],
       [0.61683565, 0.3831644 ]], dtype=float32)

In [94]:
print(result)

[[0.45764974 0.5423503 ]
 [0.55135643 0.44864354]
 [0.03857527 0.96142477]
 ...
 [0.62677145 0.37322855]
 [0.6745697  0.32543024]
 [0.61683565 0.3831644 ]]


In [95]:
result_goal = result[:,1]

In [96]:
result_goal = np.where(result_goal > 0.5, 1., 0.)

In [104]:
result_goal

array([1., 0., 1., ..., 0., 0., 0.])

In [98]:
i = 0
for j in range(result_goal.size):
    if(result_goal[j] == 1):
        i+=1;
print(i)

962


In [101]:
w = pd.Series(result_goal,name='is_goal')
submission = pd.concat([pd.Series(datatt['shot_id_number'].values,name="shot_id_number"),w],axis = 1)
submission.to_csv("sub.csv",index = False)

In [102]:
pred = pd.read_csv('D:/Dcode/sub.csv')

In [103]:
pred

,shot_id_number,is_goal
0,26001.0,1.0
1,26004.0,0.0
2,26005.0,1.0
3,26007.0,1.0
4,26008.0,0.0
5,26010.0,1.0
6,26011.0,0.0
7,26012.0,0.0
8,26013.0,1.0
9,26014.0,1.0
